In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime

def create_tensorboard_callback(folder_name):
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "-" + folder_name
    return tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [0]:
import sys
print(sys.version)

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [0]:
def starting_model(num_classes):
    return tf.keras.models.Sequential(layers=[
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), padding='valid', kernel_initializer='he_normal'), #26x26x10
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='valid', kernel_initializer='he_normal'),  #24x24x20
        tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'),   
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'),  
        tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]), 'starting', '--'

In [0]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                  min_delta=0.0001,
                                                  patience=3)

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
def create_image_generator():
  return ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15, # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [0]:
def prepare(df, is_training=True):
    X = None
    y = None

    if is_training:
      X = df.iloc[:,1:]
      y = df.iloc[:,0]
    else:
      X = df

    # normalizing input from [0,255] to [0,1]
    X = X / 255.0

    # each row is image of 28x28, one channel.
    X = np.reshape(X.values, (X.shape[0], 28, 28, 1))
    if is_training:
      y = y.values

      num_classes = 10
      y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    
    return X,y

In [0]:
batch_size = 128
epochs = 30
lossfn = 'categorical_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [0]:
def train_model(df, name):
    X,y = prepare(df)

    models = [starting_model(10)]

    losses = []
    accuracies = []

    for model, model_name, line_style in models:
        model.compile(loss=lossfn,
              optimizer=optimizer,
              metrics=['accuracy'])

        print(model.summary())

        X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.2, random_state=69420)
        callbacks=[learning_rate_reduction, create_tensorboard_callback(name)]
        datagen = create_image_generator()
        datagen.fit(X_train)
        #model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks)
        model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction, create_tensorboard_callback(name)],)

    return model


In [0]:
def train_predict(df_train, df_test):
  X_train_all, y_train_all = prepare(df_train)
  X_test_all, _ = prepare(df_test, is_training=False)

  num_classes = 10
  model, _, _ = starting_model(num_classes=num_classes)

  model.compile(loss=lossfn,
        optimizer=optimizer,
        metrics=['accuracy'])
  
  X_train, X_val, Y_train, Y_val = train_test_split(X_train_all, y_train_all, test_size = 0.2, random_state=69420)

  callbacks=[learning_rate_reduction]
  datagen = create_image_generator()
  datagen.fit(X_train)
  #model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks, validation_split=0.2)
  model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=callbacks,)

  y_pred = model.predict(X_test_all)
  y_pred_classes = np.argmax(y_pred, axis=1)

  submissions = pd.DataFrame({"ImageId": list(range(1,len(y_pred_classes)+1)), "Label": y_pred_classes})
  submissions.to_csv("submission.csv", index=False, header=True)


In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [0]:
train_model(df_train, 'base5lyr_lrred_henormal_withimggen')

In [31]:
train_predict(df_train, df_test)

Epoch 1/30
262/262 [==============================] - 9s 33ms/step - loss: 0.7335 - accuracy: 0.7677 - val_loss: 0.1181 - val_accuracy: 0.9735 - lr: 1.0000e-04
Epoch 2/30
262/262 [==============================] - 9s 33ms/step - loss: 0.2453 - accuracy: 0.9290 - val_loss: 0.0632 - val_accuracy: 0.9818 - lr: 1.0000e-04
Epoch 3/30
262/262 [==============================] - 9s 35ms/step - loss: 0.1830 - accuracy: 0.9446 - val_loss: 0.0519 - val_accuracy: 0.9844 - lr: 1.0000e-04
Epoch 4/30
262/262 [==============================] - 9s 35ms/step - loss: 0.1426 - accuracy: 0.9584 - val_loss: 0.0429 - val_accuracy: 0.9871 - lr: 1.0000e-04
Epoch 5/30
262/262 [==============================] - 9s 34ms/step - loss: 0.1271 - accuracy: 0.9619 - val_loss: 0.0428 - val_accuracy: 0.9863 - lr: 1.0000e-04
Epoch 6/30
262/262 [==============================] - 9s 33ms/step - loss: 0.1131 - accuracy: 0.9671 - val_loss: 0.0418 - val_accuracy: 0.9867 - lr: 1.0000e-04
Epoch 7/30
262/262 [====================

In [0]:
df_train.dtypes

In [0]:
import os
os.getcwd()

In [0]:
!unzip sample_data/mnist.zip

In [0]:
!zip -r tensorboardlogs.zip logs/

In [0]:
# cleaning up tensorboard logs folder
!rm -r logs/fit/

In [0]:
!rm tensorboardlogs.zip